In [ ]:
# Import necessary libraries
from langchain_google_vertexai import ChatVertexAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
import google.auth
import nest_asyncio
import asyncio
import time
import os
# Set up project info and credentials
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
credentials, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [6]:
sample_texts = [
    "Alice is a data scientist who works at Acme Corp.",
    "John is a software engineer and Alice's colleague.",
    "Acme Corp is based in San Francisco."
]

In [7]:
# Prepare sample text chunks
chunk_docs = [Document(page_content=text) for text in sample_texts]

# Define allowed nodes and relationships
# allowed_nodes = None
# allowed_relationships = None
allowed_nodes = ["Person", "Organization", "Location"]
allowed_relationships = [("Person", "works_at", "Organization"), ("Organization", "based_in", "Location")]


### Comparing models for time taken to extract graph informationm

In [8]:
def run_graph_transformer(model_name, credentials, project, location, chunk_docs, allowed_nodes, allowed_relationships,ignore_tool_usage):
    nest_asyncio.apply()
    llm = ChatVertexAI(
        model_name=model_name,
        credentials=credentials,
        project=project,
        location=location,
        temperature=0,
    )
    graph_transformer = LLMGraphTransformer(
        llm=llm,
        node_properties=False,
        relationship_properties=False,
        allowed_nodes=allowed_nodes,
        allowed_relationships=allowed_relationships,
        ignore_tool_usage=ignore_tool_usage,
    )
    start_time = time.time()
    graph_document_list = asyncio.get_event_loop().run_until_complete(
        graph_transformer.aconvert_to_graph_documents(chunk_docs)
    )
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Time taken to create graph: {elapsed:.2f} seconds for model {model_name}")
    for doc in graph_document_list:
        print("Nodes extracted : ", doc.nodes)
        print("Relationships extracted : ", doc.relationships)
    return elapsed

# Model names
MODEL1 = "gemini-2.5-flash"
LOCATION1="us-central1"

MODEL2 = "gemini-3-flash-preview"
LOCATION2="global"
ignore_tool_usage=False

# Run and compare
print("\n--- Running Gemini 2.5 Flash ---")
time1 = run_graph_transformer(MODEL1, credentials, PROJECT_ID, LOCATION1, chunk_docs, allowed_nodes, allowed_relationships,ignore_tool_usage)

print("\n--- Running Gemini 3 Preview ---")
time2 = run_graph_transformer(MODEL2, credentials, PROJECT_ID, LOCATION2, chunk_docs, allowed_nodes, allowed_relationships,ignore_tool_usage)

print(f"\nSummary:\nGemini 2.5 Flash: {time1:.2f} seconds\nGemini 3 Preview: {time2:.2f} seconds")



--- Running Gemini 2.5 Flash ---
Time taken to create graph: 3.31 seconds for model gemini-2.5-flash
Nodes extracted :  [Node(id='Alice', type='Person', properties={}), Node(id='Acme Corp.', type='Organization', properties={})]
Relationships extracted :  [Relationship(source=Node(id='Alice', type='Person', properties={}), target=Node(id='Acme Corp.', type='Organization', properties={}), type='WORKS_AT', properties={})]
Nodes extracted :  []
Relationships extracted :  []
Nodes extracted :  [Node(id='Acme Corp', type='Organization', properties={}), Node(id='San Francisco', type='Location', properties={})]
Relationships extracted :  [Relationship(source=Node(id='Acme Corp', type='Organization', properties={}), target=Node(id='San Francisco', type='Location', properties={}), type='BASED_IN', properties={})]

--- Running Gemini 3 Preview ---
Time taken to create graph: 9.46 seconds for model gemini-3-flash-preview
Nodes extracted :  [Node(id='Alice', type='Person', properties={}), Node(id=

##### Error when ignore tools usage is True

In [ ]:
sample_texts = [
    "Alice is a data scientist who works at Acme Corp.",
    "John is a software engineer and Alice's colleague.",
    "Acme Corp is based in San Francisco."
]

def run_graph_transformer(model_name, credentials, project, location, chunk_docs, allowed_nodes, allowed_relationships,ignore_tool_usage):
    nest_asyncio.apply()
    llm = ChatVertexAI(
        model_name=model_name,
        credentials=credentials,
        project=project,
        location=location,
        temperature=0,
    )
    graph_transformer = LLMGraphTransformer(
        llm=llm,
        node_properties=False,
        relationship_properties=False,
        allowed_nodes=allowed_nodes,
        allowed_relationships=allowed_relationships,
        ignore_tool_usage=ignore_tool_usage,
    )
    start_time = time.time()
    graph_document_list = asyncio.get_event_loop().run_until_complete(
        graph_transformer.aconvert_to_graph_documents(chunk_docs)
    )
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Time taken to create graph: {elapsed:.2f} seconds for model {model_name}")
    for doc in graph_document_list:
        print("Nodes extracted : ", doc.nodes)
        print("Relationships extracted : ", doc.relationships)
    return elapsed

# Model names
MODEL2 = "gemini-3-flash-preview"
LOCATION2="global"
ignore_tool_usage=True


print("\n--- Running Gemini 3 Preview ---")
time2 = run_graph_transformer(MODEL2, credentials, PROJECT_ID, LOCATION2, chunk_docs, allowed_nodes, allowed_relationships,ignore_tool_usage)

print(f"\nSummary:\nGemini 3 Preview: {time2:.2f} seconds")



--- Running Gemini 3 Preview ---


TypeError: the JSON object must be str, bytes or bytearray, not list